In [ ]:
# LIBRARY
!uv pip install -q --system numba-cuda==0.4.0
import numpy as np

import time
import os

# Enable the CUDA simulator. This MUST be set BEFORE numba imports or kernel definitions.
os.environ["NUMBA_ENABLE_CUDASIM"] = "1"
from numba import cuda
from numba import config

# --- Configuration & Data Preparation ---

config.CUDA_ENABLE_PYNVJITLINK = 1

# Prepare character data (ASCII values for A-H, 8 characters total)
characters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
data = np.array([ord(c) for c in characters], dtype=np.uint8)
data_size = len(data) # 8 elements

# --- 1D Kernel Definition (using gidx, bidx, tidx) ---

@cuda.jit
def kernel_1d_dims(arr):
    # gidx: Global ID index (Thread ID in the entire grid)
    gidx = cuda.grid(1)

    # bidx: Block ID (Block index in the grid)
    bidx = cuda.blockIdx.x
    # tidx: Thread ID (Thread index within the block)
    tidx = cuda.threadIdx.x

    if gidx < arr.size:
        # Standard Python print works via the simulator
        print(f"BID: {bidx}, TID: {tidx}, GID: {gidx}, Char: {chr(arr[gidx])}")

In [ ]:
# EXAMPLE 1: OPERATIONS WITH THREADS AND BLOCKS

blocks_per_grid_ex1 = 2
threads_per_block_ex1 = 2
# Total threads = 2 x 2 = 4

kernel_1d_dims[blocks_per_grid_ex1, threads_per_block_ex1](data)
cuda.synchronize()

In [ ]:
# EXAMPLE 2: OPERATIONS WITH THREADS AND BLOCKS

blocks_per_grid_ex1 = 2
threads_per_block_ex1 = 8
# Total threads = 2 x 8 = 16

kernel_1d_dims[blocks_per_grid_ex1, threads_per_block_ex1](data)
cuda.synchronize()

In [ ]:
# EXAMPLE 3: OPERATIONS WITH THREADS AND BLOCKS

@cuda.jit
def whoami():
    # Compute block id in a 3D grid
    block_id = (
        cuda.blockIdx.x +
        cuda.blockIdx.y * cuda.gridDim.x +
        cuda.gridDim.x * cuda.gridDim.y
    )

    # Threads per block
    threads_per_block = (
        cuda.blockDim.x * cuda.blockDim.y
    )

    # Offset of this block
    block_offset = block_id * threads_per_block

    # Compute thread id inside block
    thread_offset = (
        cuda.threadIdx.x +
        cuda.threadIdx.y * cuda.blockDim.x +
        cuda.blockDim.x * cuda.blockDim.y
    )

    # Global thread id across all blocks
    global_id = block_offset + thread_offset


    print(f"{global_id:03d} | Block[x, y]({cuda.blockIdx.x} {cuda.blockIdx.y}) = {block_id:3d} | "
          f"Thread[x, y] ({cuda.threadIdx.x} {cuda.threadIdx.y} ) = {thread_offset:3d} BlockDim.x {cuda.blockDim.x} BlockDim.y {cuda.blockDim.y} GridDim.x {cuda.gridDim.x} GridDim.y {cuda.gridDim.y}")


b_x, b_y = 2, 2
t_x, t_y = 4, 1

blocks_per_grid = (b_x, b_y)
threads_per_block = (t_x, t_y)

total_blocks = b_x * b_y
total_threads = t_x * t_y
print(f"{total_blocks} blocks/grid")
print(f"{total_threads} threads/block")
print(f"{total_blocks * total_threads} total threads\n")

# Launch kernel
whoami[blocks_per_grid, threads_per_block]()

# Wait for GPU to finish (like cudaDeviceSynchronize)
cuda.synchronize()